# Neural Networks in PyTorch
## Part 11: Training a NN using the GPU
*Yen Lee Loh, 2021-9-8, 2022-11-23*

---
## 1. Setup

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
from torch import nn        # import torch.nn as nn
import torchvision.datasets # In functional.py: patch PILLOW_VERSION--> __version__
import torchvision.transforms
from collections.abc import Iterable
rng = np.random.default_rng()
np.set_printoptions (linewidth=300)
plt.rcParams.update ({'font.family':'serif', 'font.size':13})

'''
  gallery(xnij)
  Display an array of grayscale images 
'''
def gallery(xnij, cmap='viridis', labels=None, size=1):  # size is in inches
  nmax = len(xnij)
  cols = min(20,nmax) ; rows = (nmax+cols-1)//cols
  wspace = 0.02 ; hspace = 0.02
  if isinstance (labels, Iterable) or labels!=None:   # if user has supplied labels
    hspace = .35
    
  fig,axs = plt.subplots (rows,cols, figsize=(cols*size*(1+wspace),rows*size*(1+hspace)), gridspec_kw={'wspace':wspace,'hspace':hspace})
  if nmax==1: axs = np.array([[axs]])
  axs = axs.flatten()
  for ax in axs:
      ax.axis ('off')
  for n in range(nmax):
      ax = axs[n]
      if isinstance (cmap, Iterable) and not isinstance (cmap, str):
        c = cmap[n]
      else:
        c = cmap
      ax.imshow (xnij[n], cmap=c)
      ax.set_aspect('equal')
      if isinstance (labels, Iterable):
        ax.set_title (str(labels[n]))

'''
    totalExamples,misclassifiedExamples,confusionMatrix = metrics (modelOutputs, trueOutputs)
'''
def metrics (Yn, yn):
  nmax = len(yn)
  ymax = max(yn)+1
  confmat = np.zeros ([ymax, ymax], dtype=int)   # confmat[Y][y]
  for n in range(nmax): confmat[yn[n], Yn[n]] += 1
  ntot = np.sum(confmat)
  nerr = ntot - np.trace(confmat)
  return ntot,nerr,confmat

'''
    xT,yT,xV,yV = select (MNISTinputs, MNISToutputs, [5,6,8], [100,100,100], [200,200,200])

    Given a set of inputs and outputs, 
    construct a training set consisting of the first 100 5's, 100 6's, 100 8's, 
    and a validation set consisting of the next 200 5's, 200 6's, and 200 8's.
    If the original set has fewer than 300 5's, 6's, or 7's, raise an exception.
'''
def select (inputs, outputs, classes, nT, nV, shuffle=False):
  assert len(classes) == len(nT) and len(nT) == len(nV)
  allT = []
  allV = []
  for k in range(len(classes)):
    indices, = np.where(outputs==classes[k])
    ntot = len(indices)
    indices = rng.choice (indices, nT[k] + nV[k], False)    # randomly choose 300
    indicesT,indicesV = np.split (indices, [nT[k]])
    allT += indicesT.tolist()
    allV += indicesV.tolist()
    print ('For class {}, given {} examples, we chose {} for training and {} for validation. '.format(classes[k], ntot, len(indicesT), len(indicesV)))
  if shuffle:
    rng.shuffle (allT)
    rng.shuffle (allV)
  return inputs[allT], outputs[allT], inputs[allV], outputs[allV]

In [7]:
#================ DOWNLOAD THE MNIST-TRAIN DATASET, WHICH CONTAINS 60000 HANDWRITTEN DIGITS
dataset = torchvision.datasets.MNIST('MNIST-TRAIN', download=True, train=True, transform=torchvision.transforms.ToTensor())
loader = torch.utils.data.DataLoader(dataset, batch_size=70000, shuffle=False)
iterator = iter(loader)
inputs,outputs = next(iterator)  # new PyTorch syntax; old syntax was iterator.next()
inputs = inputs.squeeze()        # get rid of unnecessary dimension

In [8]:
print ("torch.cuda.is_available() =", torch.cuda.is_available())

torch.cuda.is_available() = True


---
## 2. Load a digit pair from the MNIST dataset

In [14]:
class0,class1 = 3,8     #1,5
xnijT,ynT,xnijV,ynV = select (inputs, outputs, [class0,class1], [2500,2500], [2500,2500], shuffle=True) 
xndT = xnijT.flatten (1,-1) ; yndT = (ynT.reshape (-1,1) - class0) / (class1-class0)
xndV = xnijV.flatten (1,-1) ; yndV = (ynV.reshape (-1,1) - class0) / (class1-class0)
_,imax,jmax = xnijT.shape
_,dmax = xndT.shape

For class 3, given 6131 examples, we chose 2500 for training and 2500 for validation. 
For class 8, given 5851 examples, we chose 2500 for training and 2500 for validation. 


---
## 3. Train SLP on CPU and on GPU

In [37]:
def train(xnd, ynd, model, lossFunc, epochs=10000, learningRate=0.01, lossTarget=0.0001, reportInterval=1000):
  optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
  model.train()                  # put model in training mode
  for t in range(epochs):      # t is the epoch number
    Ynd = model(xnd)             # uppercase Y = model prediction
    loss = lossFunc(Ynd,ynd)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if t % reportInterval == 0 or t==epochs:
      F = loss.item()
      print('Training epoch {}/{}  \t Loss = {:.4f}'.format(t, epochs, F))
      if F < lossTarget:
        print('Training epoch {}/{}  \t Loss = {:.4f} < lossTarget\n'.format(t, epochs, F))
        return
  print ('Warning: loss > lossTarget!\n')

In [38]:
#================ TRAIN MODEL ON CPU
model    = nn.Sequential(nn.Linear(dmax,1),nn.Sigmoid())
lossFunc = nn.BCELoss()
train (xndT, yndT, model, lossFunc)

Training epoch 0/10000  	 Loss = 0.7005
Training epoch 1000/10000  	 Loss = 0.0540
Training epoch 2000/10000  	 Loss = 0.0417
Training epoch 3000/10000  	 Loss = 0.0335
Training epoch 4000/10000  	 Loss = 0.0276
Training epoch 5000/10000  	 Loss = 0.0226
Training epoch 6000/10000  	 Loss = 0.0182
Training epoch 7000/10000  	 Loss = 0.0142
Training epoch 8000/10000  	 Loss = 0.0107
Training epoch 9000/10000  	 Loss = 0.0080



In [39]:
#================ EVALUATE ACCURACY FOR BOTH TRAINING AND VALIDATION SETS
model.eval()             # choose evaluation mode
YndT = model(xndT)
YndV = model(xndV)
YnT = YndT.detach().numpy().round().flatten().astype(int)  # round to either 0 or 1
ynT = yndT.detach().numpy().flatten().astype(int)          # this is already an integer
YnV = YndV.detach().numpy().round().flatten().astype(int)  # round to either 0 or 1
ynV = yndV.detach().numpy().flatten().astype(int)          # this is already an integer

ntot,nerr,Cnn = metrics (YnT, ynT)
print("Training error   = {:4d}/{:} = {:4.1f}%    Confusion matrix = {}".format (nerr, ntot, 100*nerr/ntot, Cnn.tolist()))
ntot,nerr,Cnn = metrics (YnV, ynV)
print("Validation error = {:4d}/{:} = {:4.1f}%    Confusion matrix = {}".format (nerr, ntot, 100*nerr/ntot, Cnn.tolist()))


Training error   =    0/5000 =  0.0%    Confusion matrix = [[2500, 0], [0, 2500]]
Validation error =  320/5000 =  6.4%    Confusion matrix = [[2345, 155], [165, 2335]]


In [40]:
#================ TRAIN MODEL ON GPU INSTEAD
model    = nn.Sequential(nn.Linear(dmax,1),nn.Sigmoid())
lossFunc = nn.BCELoss()
XNDT = xndT.cuda()
YNDT = yndT.cuda()
MODEL = model.cuda()
LOSSF = lossFunc.cuda()
train (XNDT, YNDT, MODEL, LOSSF)

Training epoch 0/10000  	 Loss = 0.6998
Training epoch 1000/10000  	 Loss = 0.0529
Training epoch 2000/10000  	 Loss = 0.0406
Training epoch 3000/10000  	 Loss = 0.0327
Training epoch 4000/10000  	 Loss = 0.0268
Training epoch 5000/10000  	 Loss = 0.0219
Training epoch 6000/10000  	 Loss = 0.0176
Training epoch 7000/10000  	 Loss = 0.0138
Training epoch 8000/10000  	 Loss = 0.0104
Training epoch 9000/10000  	 Loss = 0.0078



In [41]:
model = MODEL.cpu()
YndT = YNDT.cpu()

In [42]:
#================ EVALUATE ACCURACY FOR BOTH TRAINING AND VALIDATION SETS
model.eval()             # choose evaluation mode
YndT = model(xndT)
YndV = model(xndV)
YnT = YndT.detach().numpy().round().flatten().astype(int)  # round to either 0 or 1
ynT = yndT.detach().numpy().flatten().astype(int)          # this is already an integer
YnV = YndV.detach().numpy().round().flatten().astype(int)  # round to either 0 or 1
ynV = yndV.detach().numpy().flatten().astype(int)          # this is already an integer

ntot,nerr,Cnn = metrics (YnT, ynT)
print("Training error   = {:4d}/{:} = {:4.1f}%    Confusion matrix = {}".format (nerr, ntot, 100*nerr/ntot, Cnn.tolist()))
ntot,nerr,Cnn = metrics (YnV, ynV)
print("Validation error = {:4d}/{:} = {:4.1f}%    Confusion matrix = {}".format (nerr, ntot, 100*nerr/ntot, Cnn.tolist()))


Training error   =    0/5000 =  0.0%    Confusion matrix = [[2500, 0], [0, 2500]]
Validation error =  320/5000 =  6.4%    Confusion matrix = [[2345, 155], [165, 2335]]


---
## 4. Train CNN on CPU and on GPU

In [47]:
#======== TRAIN
model = nn.Sequential(               # 28x28 ?
  nn.Unflatten (1, (1,28,28)),       # 1x28x28
  nn.Conv2d    (1, 8, 5, padding=2), # 1x28x28 (after conv with 5x5 kernel and 2x2 padding)
  nn.ReLU(),
  nn.MaxPool2d (2),                  # 1x14x14 after pooling layer with 2x2 kernel
  nn.Conv2d    (8, 16, 5),           # 6x10x10 after conv with 5x5 kernel and no padding
  nn.ReLU(),
  nn.MaxPool2d (2),                  # 16x5x5
  nn.Flatten(),    
  nn.Linear(16*5*5,84),              # 84
  nn.ReLU(),
  nn.Linear(84,1),                   # 1
  nn.Sigmoid()
)

In [48]:
lossFunc = nn.BCELoss()
XNDT = xndT.cuda()
YNDT = yndT.cuda()
MODEL = model.cuda()
LOSSF = lossFunc.cuda()
train (XNDT, YNDT, MODEL, LOSSF, reportInterval=20)

Training epoch 0/10000  	 Loss = 0.6933
Training epoch 20/10000  	 Loss = 0.0739
Training epoch 40/10000  	 Loss = 0.0214
Training epoch 60/10000  	 Loss = 0.0079
Training epoch 80/10000  	 Loss = 0.0021
Training epoch 100/10000  	 Loss = 0.0006
Training epoch 120/10000  	 Loss = 0.0002
Training epoch 140/10000  	 Loss = 0.0001
Training epoch 160/10000  	 Loss = 0.0001
Training epoch 160/10000  	 Loss = 0.0001 < lossTarget



In [49]:
model = MODEL.cpu()
YndT = YNDT.cpu()

In [50]:
#================ EVALUATE ACCURACY FOR BOTH TRAINING AND VALIDATION SETS
model.eval()             # choose evaluation mode
YndT = model(xndT)
YndV = model(xndV)
YnT = YndT.detach().numpy().round().flatten().astype(int)  # round to either 0 or 1
ynT = yndT.detach().numpy().flatten().astype(int)          # this is already an integer
YnV = YndV.detach().numpy().round().flatten().astype(int)  # round to either 0 or 1
ynV = yndV.detach().numpy().flatten().astype(int)          # this is already an integer

ntot,nerr,Cnn = metrics (YnT, ynT)
print("Training error   = {:4d}/{:} = {:4.1f}%    Confusion matrix = {}".format (nerr, ntot, 100*nerr/ntot, Cnn.tolist()))
ntot,nerr,Cnn = metrics (YnV, ynV)
print("Validation error = {:4d}/{:} = {:4.1f}%    Confusion matrix = {}".format (nerr, ntot, 100*nerr/ntot, Cnn.tolist()))


Training error   =    0/5000 =  0.0%    Confusion matrix = [[2500, 0], [0, 2500]]
Validation error =   27/5000 =  0.5%    Confusion matrix = [[2491, 9], [18, 2482]]
